In [3]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Dash config with parent asset path
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🧍 Human Face Carousel"

# 📦 Paths
HUMAN_PATH = os.path.abspath("../assets/human")
HUMAN_PARTS = sorted([d for d in os.listdir(HUMAN_PATH) if os.path.isdir(f"{HUMAN_PATH}/{d}")])

# 📦 Face-specific subfolders
FACE_SEX = ["female", "male"]
FACE_AGES = ["baby", "child", "adolescent", "adult", "elder"]

# 📦 Load non-face images
basic_images = {
    part: sorted([
        f"/assets/human/{part}/{img}"
        for img in os.listdir(os.path.join(HUMAN_PATH, part))
        if img.lower().endswith((".jpg", ".jpeg", ".png"))
    ])
    for part in HUMAN_PARTS if part != "face"
}

# 📦 Load face images
face_images = {}
for sex in FACE_SEX:
    for age in FACE_AGES:
        folder = os.path.join(HUMAN_PATH, "face", sex, age)
        if os.path.isdir(folder):
            face_images[(sex, age)] = sorted([
                f"/assets/human/face/{sex}/{age}/{img}"
                for img in os.listdir(folder)
                if img.lower().endswith((".jpg", ".jpeg", ".png"))
            ])

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🧍 Human Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "28px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select body part:", style={
            "fontWeight": "normal", "display": "block", "marginBottom": "6px",
            "fontSize": "15px", "fontFamily": "Georgia, serif"
        }),
        dcc.Dropdown(
            id='part-dropdown',
            options=[{"label": part.capitalize(), "value": part} for part in HUMAN_PARTS],
            value=HUMAN_PARTS[0] if HUMAN_PARTS else None,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='sub-dropdowns', style={"textAlign": "center", "marginBottom": "20px"}),
    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Update sex/age dropdowns if 'face'
@app.callback(
    Output('sub-dropdowns', 'children'),
    Input('part-dropdown', 'value')
)
def update_sub_dropdowns(part):
    if part == "face":
        return html.Div([
            html.Label("Sex:", style={"marginBottom": "6px", "display": "block"}),
            dcc.Dropdown(
                id='sex-dropdown',
                options=[{"label": s.capitalize(), "value": s} for s in FACE_SEX],
                value=FACE_SEX[0],
                clearable=False,
                style={"width": "200px", "margin": "0 auto 12px auto"}
            ),
            html.Label("Age group:", style={"marginBottom": "6px", "display": "block"}),
            dcc.Dropdown(
                id='age-dropdown',
                options=[{"label": a.capitalize(), "value": a} for a in FACE_AGES],
                value=FACE_AGES[0],
                clearable=False,
                style={"width": "200px", "margin": "0 auto"}
            )
        ])
    else:
        return html.Div([
            dcc.Dropdown(id='sex-dropdown', value='female', style={'display': 'none'}),
            dcc.Dropdown(id='age-dropdown', value='baby', style={'display': 'none'})
        ])

# 🔁 Update carousel based on dropdowns
@app.callback(
    Output('carousel-container', 'children'),
    Input('part-dropdown', 'value'),
    Input('sex-dropdown', 'value'),
    Input('age-dropdown', 'value'),
    prevent_initial_call=False
)
def update_carousel(part, sex, age):
    if part == "face":
        images = face_images.get((sex, age), [])
    else:
        images = basic_images.get(part, [])

    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginRight": "20px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginLeft": "20px"}),
        ], style={"textAlign": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontStyle": "italic", "fontFamily": "Georgia, serif"})
    ])

# 🔁 Navigation logic
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Run the app
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8163)
